[View in Colaboratory](https://colab.research.google.com/github/guoyaohua/DSR/blob/master/SCNN_Model.ipynb)

In [1]:
cd drive/DSR/src

/content/drive/DSR/src


In [6]:
# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""
#%%
import tensorflow as tf
from keras.utils import np_utils
import numpy as np
import GetDataUtil
import time

'''
定义超参
'''
BATCH_SIZE = 128 # 批大小
EPOCH = 5 # 训练EPOCH次数
HIDDEN_UNIT = 512 
KERNEL_SIZE = 3 # 通用卷集核大小
COVN_1_CHANNELS = 32 # 第一层卷积层的输出chennel个数
COVN_2_CHANNELS = 16 # 第二层卷积层的输出chennel个数
INPUT_CHENNELS = 3 # 输入图片的chennels个数
SCNN_KERNEL_LENGTH = 9 # SCNN 卷集核的宽度
PAD = SCNN_KERNEL_LENGTH - 1 # 切片后需要pad的个数
# 切片需要后需要PAD
PADDING = [[0, 0],
           [0,0],
           [int(PAD / 2),PAD - int(PAD/2)],       
           [0,0]]

BATCH_SIZE = 128
CONV1_FILTERS = 64
CONV2_FILTERS = 32

'''
数据读取
'''
x_train, X_test, y_train, y_test = GetDataUtil.getTrainTestSet(dataPath = "../DataSet_NPSave/JustifiedData.npy",test_size = 0.1)

x = x_train[:,:,:,np.newaxis]
y_onehot = np_utils.to_categorical(y_train-1,5)

x_test = X_test[:,:,:,np.newaxis]
y_test = np_utils.to_categorical(y_test-1,5)


#x = np.random.randn(100,38,38,1).astype(np.float32)
#y = np.random.randint(1,5,100,dtype = np.int32)
#print(y_onehot)

#%% SCNN 模块生成部分
'''
SCNN_Down 向下传递信息
'''
def SCNN_D(SCNN_D_input):
     CHANNELS = SCNN_D_input.shape[3]
     FEATURE_MAP_H = SCNN_D_input.shape[1]
     # 声明参数
     w = tf.get_variable(name = 'w_scnn_d',
                         shape = [CHANNELS,
                                  SCNN_KERNEL_LENGTH,
                                  1,
                                  CHANNELS],
                         initializer = tf.keras.initializers.glorot_normal(),
                         regularizer = tf.keras.regularizers.l2())
     b = tf.Variable(tf.zeros(CHANNELS),
                     name = "b_scnn_d")
     
     # 用于动态生成网络层
     SCNN_D = {}
   
     '''
     状态释义：
     state = 1：(when i == 0 && H > 2)
          无slice_top，存在slice_conv,slice_add,slice_bottom
     state = 2: (when i == H-2 && i > 0)
          无slice_bottom，存在slice_top,slice_conv,slice_add
     state = 3: (when i == H-1 && i > 0)
          无slice_add,slice_bottom，存在slice_top,slice_conv
     state = 4: (when H == 1)
          无slice_top,slice_add,slice_bottom，存在slice_conv
     state = 5: (when i == 0 && H == 2)
          无slice_top,slice_bottom，存在slice_conv,slice_add
     state = 6:
          存在slice_top,slice_conv,slice_add,slice_bottom
     '''
     for i in range(FEATURE_MAP_H):
          # State 1 : 无slice_top，存在slice_conv,slice_add,slice_bottom 
          if i == 0 and FEATURE_MAP_H > 2:
               SCNN_D['slice_conv_'+str(i)] = tf.slice(SCNN_D_input,
                                                      begin = [0, i, 0, 0],
                                                      size = [-1, 1, -1, -1])
               SCNN_D['slice_add_'+str(i)] = tf.slice(SCNN_D_input,
                                                      begin = [0, i+1, 0, 0],
                                                      size = [-1, 1, -1, -1])
               SCNN_D['slice_bottom_'+str(i)] = tf.slice(SCNN_D_input,
                                                      begin = [0, i+2, 0, 0],
                                                      size = [-1, -1, -1, -1])
               SCNN_D['tran_bef_'+str(i)] = tf.transpose(SCNN_D['slice_conv_'+str(i)],
                                                         perm = [0,3,2,1])
               SCNN_D['pad_'+str(i)] = tf.pad(SCNN_D['tran_bef_'+str(i)],
                                              paddings = PADDING,
                                              mode='CONSTANT',
                                              constant_values=0)
               SCNN_D['conv_'+str(i)] = tf.nn.conv2d(SCNN_D['pad_'+str(i)],
                                                     filter = w,
                                                     strides = [1, 1, 1, 1],
                                                     padding = 'VALID',
                                                     use_cudnn_on_gpu=True,
                                                     data_format='NHWC',
                                                     dilations=[1, 1, 1, 1])
               SCNN_D['conv_bias_relu_'+str(i)] = tf.nn.relu(tf.nn.bias_add(SCNN_D['conv_'+str(i)],b),
                                                             name = None)               
               SCNN_D['add_'+str(i)] = tf.add(SCNN_D['conv_bias_relu_'+str(i)],
                                              SCNN_D['slice_add_'+str(i)])
               SCNN_D['concat_'+str(i)] = tf.concat(values = [SCNN_D['conv_bias_relu_'+str(i)],
                                                              SCNN_D['add_'+str(i)],
                                                              SCNN_D['slice_bottom_'+str(i)]],axis = 1)
          # State 2 : 无slice_bottom，存在slice_top,slice_conv,slice_add 
          elif i == FEATURE_MAP_H - 2 and i > 0:
               SCNN_D['slice_top_'+str(i)] = tf.slice(SCNN_D['concat_'+str(i-1)],
                                                      begin = [0, 0, 0, 0],
                                                      size = [-1, i, -1, -1])
               SCNN_D['slice_conv_'+str(i)] = tf.slice(SCNN_D['concat_'+str(i-1)],
                                                      begin = [0, i, 0, 0],
                                                      size = [-1, 1, -1, -1])
               SCNN_D['slice_add_'+str(i)] = tf.slice(SCNN_D['concat_'+str(i-1)],
                                                      begin = [0, i+1, 0, 0],
                                                      size = [-1, 1, -1, -1])
               SCNN_D['tran_bef_'+str(i)] = tf.transpose(SCNN_D['slice_conv_'+str(i)],
                                                         perm = [0,3,2,1])
               SCNN_D['pad_'+str(i)] = tf.pad(SCNN_D['tran_bef_'+str(i)],
                                              paddings = PADDING,
                                              mode='CONSTANT',
                                              constant_values=0)
               SCNN_D['conv_'+str(i)] = tf.nn.conv2d(SCNN_D['pad_'+str(i)],
                                                     filter = w,
                                                     strides = [1, 1, 1, 1],
                                                     padding = 'VALID',
                                                     use_cudnn_on_gpu=True,
                                                     data_format='NHWC',
                                                     dilations=[1, 1, 1, 1])
               SCNN_D['conv_bias_relu_'+str(i)] = tf.nn.relu(tf.nn.bias_add(SCNN_D['conv_'+str(i)],b),
                                                             name = None)
               SCNN_D['add_'+str(i)] = tf.add(SCNN_D['conv_bias_relu_'+str(i)],
                                              SCNN_D['slice_add_'+str(i)])
               SCNN_D['concat_'+str(i)] = tf.concat(values = [SCNN_D['slice_top_'+str(i)],
                                                              SCNN_D['conv_bias_relu_'+str(i)],
                                                              SCNN_D['add_'+str(i)]],axis = 1)
          # State 3 : 无slice_add,slice_bottom，存在slice_top,slice_conv 
          elif i == FEATURE_MAP_H - 1 and i > 0:
               SCNN_D['slice_top_'+str(i)] = tf.slice(SCNN_D['concat_'+str(i-1)],
                                                      begin = [0, 0, 0, 0],
                                                      size = [-1, i, -1, -1])
               SCNN_D['slice_conv_'+str(i)] = tf.slice(SCNN_D['concat_'+str(i-1)],
                                                      begin = [0, i, 0, 0],
                                                      size = [-1, 1, -1, -1])
               SCNN_D['tran_bef_'+str(i)] = tf.transpose(SCNN_D['slice_conv_'+str(i)],
                                                         perm = [0,3,2,1])
               SCNN_D['pad_'+str(i)] = tf.pad(SCNN_D['tran_bef_'+str(i)],
                                              paddings = PADDING,
                                              mode='CONSTANT',
                                              constant_values=0)
               SCNN_D['conv_'+str(i)] = tf.nn.conv2d(SCNN_D['pad_'+str(i)],
                                                     filter = w,
                                                     strides = [1, 1, 1, 1],
                                                     padding = 'VALID',
                                                     use_cudnn_on_gpu=True,
                                                     data_format='NHWC',
                                                     dilations=[1, 1, 1, 1],
                                                     name=None)
               SCNN_D['conv_bias_relu_'+str(i)] = tf.nn.relu(tf.nn.bias_add(SCNN_D['conv_'+str(i)],b),
                                                             name = None)
               SCNN_D['concat_'+str(i)] = tf.concat(values = [SCNN_D['slice_top_'+str(i)],
                                                              SCNN_D['conv_bias_relu_'+str(i)]], axis = 1)
          # State 4 : 无slice_top,slice_add,slice_bottom，存在slice_conv 
          elif FEATURE_MAP_H == 1:
               SCNN_D['slice_conv_'+str(i)] = tf.slice(SCNN_D_input,
                                                      begin = [0,i,0,0],
                                                      size = [-1,-1,-1,-1])
               SCNN_D['tran_bef_'+str(i)] = tf.transpose(SCNN_D['slice_conv_'+str(i)],
                                                         perm = [0,3,2,1])
               SCNN_D['pad_'+str(i)] = tf.pad(SCNN_D['tran_bef_'+str(i)],
                                              paddings = PADDING,
                                              mode='CONSTANT',
                                              constant_values=0)
               SCNN_D['conv_'+str(i)] = tf.nn.conv2d(SCNN_D['pad_'+str(i)],
                                                     filter = w,
                                                     strides = [1, 1, 1, 1],
                                                     padding = 'VALID',
                                                     use_cudnn_on_gpu=True,
                                                     data_format='NHWC',
                                                     dilations=[1, 1, 1, 1],
                                                     name=None)
               SCNN_D['conv_bias_relu_'+str(i)] = tf.nn.relu(tf.nn.bias_add(SCNN_D['conv_'+str(i)],b),
                                                             name = None)
               SCNN_D['concat_'+str(i)] = SCNN_D['conv_bias_relu_'+str(i)]
          # State 5 : 无slice_top,slice_bottom，存在slice_conv,slice_add
          elif i == 0 and FEATURE_MAP_H == 2:
               SCNN_D['slice_conv_'+str(i)] = tf.slice(SCNN_D_input,
                                                      begin = [0,i,0,0],
                                                      size = [-1,1,-1,-1])
               SCNN_D['slice_add_'+str(i)] = tf.slice(SCNN_D_input,
                                                      begin = [0,i+1,0,0],
                                                      size = [-1,1,-1,-1])
               SCNN_D['tran_bef_'+str(i)] = tf.transpose(SCNN_D['slice_conv_'+str(i)],
                                                         perm = [0,3,2,1])
               SCNN_D['pad_'+str(i)] = tf.pad(SCNN_D['tran_bef_'+str(i)],
                                              paddings = PADDING,
                                              mode='CONSTANT',
                                              constant_values=0)
               SCNN_D['conv_'+str(i)] = tf.nn.conv2d(SCNN_D['pad_'+str(i)],
                                                     filter = w,
                                                     strides = [1, 1, 1, 1],
                                                     padding = 'VALID',
                                                     use_cudnn_on_gpu=True,
                                                     data_format='NHWC',
                                                     dilations=[1, 1, 1, 1],
                                                     name=None)
               SCNN_D['conv_bias_relu_'+str(i)] = tf.nn.relu(tf.nn.bias_add(SCNN_D['conv_'+str(i)],b),
                                                             name = None)
               SCNN_D['add_'+str(i)] = tf.add(SCNN_D['conv_bias_relu_'+str(i)],
                                              SCNN_D['slice_add_'+str(i)])
               SCNN_D['concat_'+str(i)] = tf.concat(values = [SCNN_D['conv_bias_relu_'+str(i)],
                                                              SCNN_D['add_'+str(i)]],axis = 1)
          # State 6 : 存在slice_top,slice_conv,slice_add,slice_bottom
          else:
               SCNN_D['slice_top_'+str(i)] = tf.slice(SCNN_D['concat_'+str(i-1)],
                                                      begin = [0,0,0,0],
                                                      size = [-1,i,-1,-1])
               SCNN_D['slice_conv_'+str(i)] = tf.slice(SCNN_D['concat_'+str(i-1)],
                                                      begin = [0,i,0,0],
                                                      size = [-1,1,-1,-1])
               SCNN_D['slice_add_'+str(i)] = tf.slice(SCNN_D['concat_'+str(i-1)],
                                                      begin = [0,i+1,0,0],
                                                      size = [-1,1,-1,-1])
               SCNN_D['slice_bottom_'+str(i)] = tf.slice(SCNN_D['concat_'+str(i-1)],
                                                      begin = [0,i+2,0,0],
                                                      size = [-1,-1,-1,-1])
               SCNN_D['tran_bef_'+str(i)] = tf.transpose(SCNN_D['slice_conv_'+str(i)],
                                                         perm = [0,3,2,1])
               SCNN_D['pad_'+str(i)] = tf.pad(SCNN_D['tran_bef_'+str(i)],
                                              paddings = PADDING,
                                              mode='CONSTANT',
                                              constant_values=0)
               SCNN_D['conv_'+str(i)] = tf.nn.conv2d(SCNN_D['pad_'+str(i)],
                                                     filter = w,
                                                     strides = [1, 1, 1, 1],
                                                     padding = 'VALID',
                                                     use_cudnn_on_gpu=True,
                                                     data_format='NHWC',
                                                     dilations=[1, 1, 1, 1],
                                                     name=None)
               SCNN_D['conv_bias_relu_'+str(i)] = tf.nn.relu(tf.nn.bias_add(SCNN_D['conv_'+str(i)],b),
                                                             name = None)
               SCNN_D['add_'+str(i)] = tf.add(SCNN_D['conv_bias_relu_'+str(i)],
                                              SCNN_D['slice_add_'+str(i)])
               SCNN_D['concat_'+str(i)] = tf.concat(values = [SCNN_D['slice_top_'+str(i)],
                                                              SCNN_D['conv_bias_relu_'+str(i)],
                                                              SCNN_D['add_'+str(i)],
                                                              SCNN_D['slice_bottom_'+str(i)]],axis = 1)

     return SCNN_D['concat_'+str(FEATURE_MAP_H-1)]



'''
SCNN_Up 向上传递信息
'''
def SCNN_U(SCNN_U_input):
     CHANNELS = SCNN_U_input.shape[3]
     FEATURE_MAP_H = SCNN_U_input.shape[1]
     # 声明参数
     w = tf.get_variable(name = 'w_scnn_u',
                         shape = [CHANNELS,
                                  SCNN_KERNEL_LENGTH,
                                  1,
                                  CHANNELS],
                         initializer = tf.keras.initializers.glorot_normal(),
                         regularizer = tf.keras.regularizers.l2())
     b = tf.Variable(tf.zeros(CHANNELS),
                     name = "b_scnn_u")
     
     # 用于动态生成网络层
     SCNN_U = {}

     '''
     状态释义：
     state = 1：(when i == H-1 && H > 2)
          无slice_bottom,存在slice_conv,slice_add,slice_top
     state = 2: (when i == 1 && H > 2)
          无slice_top，存在slice_bottom,slice_conv,slice_add
     state = 3: (when i == 0 && H > 1)
          无slice_add,slice_top，存在slice_bottom,slice_conv
     state = 4: (when H == 1)
          无slice_top,slice_add,slice_bottom，存在slice_conv
     state = 5: (when i == H-1 && H == 2)
          无slice_top,slice_bottom，存在slice_conv,slice_add
     state = 6:
          存在slice_top,slice_conv,slice_add,slice_bottom
     '''
     for i in range(FEATURE_MAP_H-1,-1,-1):
          # State 1 : 无slice_bottom,存在slice_conv,slice_add,slice_top 
          if i == FEATURE_MAP_H-1 and FEATURE_MAP_H > 2:
               SCNN_U['slice_conv_'+str(i)] = tf.slice(SCNN_U_input,
                                                      begin = [0, i, 0, 0],
                                                      size = [-1, 1, -1, -1])
               SCNN_U['slice_add_'+str(i)] = tf.slice(SCNN_U_input,
                                                      begin = [0, i-1, 0, 0],
                                                      size = [-1, 1, -1, -1])
               SCNN_U['slice_top_'+str(i)] = tf.slice(SCNN_U_input,
                                                      begin = [0, 0, 0, 0],
                                                      size = [-1, i-1, -1, -1])
               SCNN_U['tran_bef_'+str(i)] = tf.transpose(SCNN_U['slice_conv_'+str(i)],
                                                         perm = [0,3,2,1])
               SCNN_U['pad_'+str(i)] = tf.pad(SCNN_U['tran_bef_'+str(i)],
                                              paddings = PADDING,
                                              mode='CONSTANT',
                                              constant_values=0)
               SCNN_U['conv_'+str(i)] = tf.nn.conv2d(SCNN_U['pad_'+str(i)],
                                                     filter = w,
                                                     strides = [1, 1, 1, 1],
                                                     padding = 'VALID',
                                                     use_cudnn_on_gpu=True,
                                                     data_format='NHWC',
                                                     dilations=[1, 1, 1, 1])
               SCNN_U['conv_bias_relu_'+str(i)] = tf.nn.relu(tf.nn.bias_add(SCNN_U['conv_'+str(i)],b),
                                                             name = None)               
               SCNN_U['add_'+str(i)] = tf.add(SCNN_U['conv_bias_relu_'+str(i)],
                                              SCNN_U['slice_add_'+str(i)])
               SCNN_U['concat_'+str(i)] = tf.concat(values = [SCNN_U['slice_top_'+str(i)],
                                                              SCNN_U['add_'+str(i)],
                                                              SCNN_U['conv_bias_relu_'+str(i)]],axis = 1)
          # State 2 : 无slice_top，存在slice_bottom,slice_conv,slice_add 
          elif i == 1 and FEATURE_MAP_H > 2:
               SCNN_U['slice_add_'+str(i)] = tf.slice(SCNN_U['concat_'+str(i+1)],
                                                      begin = [0, 0, 0, 0],
                                                      size = [-1, i, -1, -1])
               SCNN_U['slice_conv_'+str(i)] = tf.slice(SCNN_U['concat_'+str(i+1)],
                                                      begin = [0, i, 0, 0],
                                                      size = [-1, 1, -1, -1])
               SCNN_U['slice_bottom_'+str(i)] = tf.slice(SCNN_U['concat_'+str(i+1)],
                                                      begin = [0, i+1, 0, 0],
                                                      size = [-1, -1, -1, -1])
               SCNN_U['tran_bef_'+str(i)] = tf.transpose(SCNN_U['slice_conv_'+str(i)],
                                                         perm = [0,3,2,1])
               SCNN_U['pad_'+str(i)] = tf.pad(SCNN_U['tran_bef_'+str(i)],
                                              paddings = PADDING,
                                              mode='CONSTANT',
                                              constant_values=0)
               SCNN_U['conv_'+str(i)] = tf.nn.conv2d(SCNN_U['pad_'+str(i)],
                                                     filter = w,
                                                     strides = [1, 1, 1, 1],
                                                     padding = 'VALID',
                                                     use_cudnn_on_gpu=True,
                                                     data_format='NHWC',
                                                     dilations=[1, 1, 1, 1])
               SCNN_U['conv_bias_relu_'+str(i)] = tf.nn.relu(tf.nn.bias_add(SCNN_U['conv_'+str(i)],b),
                                                             name = None)
               SCNN_U['add_'+str(i)] = tf.add(SCNN_U['conv_bias_relu_'+str(i)],
                                              SCNN_U['slice_add_'+str(i)])
               SCNN_U['concat_'+str(i)] = tf.concat(values = [SCNN_U['add_'+str(i)],
                                                              SCNN_U['conv_bias_relu_'+str(i)],
                                                              SCNN_U['slice_bottom_'+str(i)]],axis = 1)
          # State 3 : 无slice_add,slice_top，存在slice_bottom,slice_conv
          elif i == 0 and FEATURE_MAP_H > 1:
               SCNN_U['slice_conv_'+str(i)] = tf.slice(SCNN_U['concat_'+str(i+1)],
                                                      begin = [0, i, 0, 0],
                                                      size = [-1, 1, -1, -1])
               SCNN_U['slice_bottom_'+str(i)] = tf.slice(SCNN_U['concat_'+str(i+1)],
                                                      begin = [0, i+1, 0, 0],
                                                      size = [-1, -1, -1, -1])
               SCNN_U['tran_bef_'+str(i)] = tf.transpose(SCNN_U['slice_conv_'+str(i)],
                                                         perm = [0,3,2,1])
               SCNN_U['pad_'+str(i)] = tf.pad(SCNN_U['tran_bef_'+str(i)],
                                              paddings = PADDING,
                                              mode='CONSTANT',
                                              constant_values=0)
               SCNN_U['conv_'+str(i)] = tf.nn.conv2d(SCNN_U['pad_'+str(i)],
                                                     filter = w,
                                                     strides = [1, 1, 1, 1],
                                                     padding = 'VALID',
                                                     use_cudnn_on_gpu=True,
                                                     data_format='NHWC',
                                                     dilations=[1, 1, 1, 1],
                                                     name=None)
               SCNN_U['conv_bias_relu_'+str(i)] = tf.nn.relu(tf.nn.bias_add(SCNN_U['conv_'+str(i)],b),
                                                             name = None)
               SCNN_U['concat_'+str(i)] = tf.concat(values = [SCNN_U['conv_bias_relu_'+str(i)],
                                                              SCNN_U['slice_bottom_'+str(i)]], axis = 1)
          # State 4 : 无slice_top,slice_add,slice_bottom，存在slice_conv 
          elif FEATURE_MAP_H == 1:
               SCNN_U['slice_conv_'+str(i)] = tf.slice(SCNN_U_input,
                                                      begin = [0,i,0,0],
                                                      size = [-1,-1,-1,-1])
               SCNN_U['tran_bef_'+str(i)] = tf.transpose(SCNN_U['slice_conv_'+str(i)],
                                                         perm = [0,3,2,1])
               SCNN_U['pad_'+str(i)] = tf.pad(SCNN_U['tran_bef_'+str(i)],
                                              paddings = PADDING,
                                              mode='CONSTANT',
                                              constant_values=0)
               SCNN_U['conv_'+str(i)] = tf.nn.conv2d(SCNN_U['pad_'+str(i)],
                                                     filter = w,
                                                     strides = [1, 1, 1, 1],
                                                     padding = 'VALID',
                                                     use_cudnn_on_gpu=True,
                                                     data_format='NHWC',
                                                     dilations=[1, 1, 1, 1],
                                                     name=None)
               SCNN_U['conv_bias_relu_'+str(i)] = tf.nn.relu(tf.nn.bias_add(SCNN_U['conv_'+str(i)],b),
                                                             name = None)
               SCNN_U['concat_'+str(i)] = SCNN_U['conv_bias_relu_'+str(i)]
          # State 5 : 无slice_top,slice_bottom，存在slice_conv,slice_add
          elif i == FEATURE_MAP_H-1 and FEATURE_MAP_H == 2:
               SCNN_U['slice_add_'+str(i)] = tf.slice(SCNN_U_input,
                                                      begin = [0,i,0,0],
                                                      size = [-1,1,-1,-1])
               SCNN_U['slice_conv_'+str(i)] = tf.slice(SCNN_U_input,
                                                      begin = [0,i+1,0,0],
                                                      size = [-1,1,-1,-1])
               SCNN_U['tran_bef_'+str(i)] = tf.transpose(SCNN_U['slice_conv_'+str(i)],
                                                         perm = [0,3,2,1])
               SCNN_U['pad_'+str(i)] = tf.pad(SCNN_U['tran_bef_'+str(i)],
                                              paddings = PADDING,
                                              mode='CONSTANT',
                                              constant_values=0)
               SCNN_U['conv_'+str(i)] = tf.nn.conv2d(SCNN_U['pad_'+str(i)],
                                                     filter = w,
                                                     strides = [1, 1, 1, 1],
                                                     padding = 'VALID',
                                                     use_cudnn_on_gpu=True,
                                                     data_format='NHWC',
                                                     dilations=[1, 1, 1, 1],
                                                     name=None)
               SCNN_U['conv_bias_relu_'+str(i)] = tf.nn.relu(tf.nn.bias_add(SCNN_U['conv_'+str(i)],b),
                                                             name = None)
               SCNN_U['add_'+str(i)] = tf.add(SCNN_U['conv_bias_relu_'+str(i)],
                                              SCNN_U['slice_add_'+str(i)])
               SCNN_U['concat_'+str(i)] = tf.concat(values = [SCNN_U['add_'+str(i)],
                                                              SCNN_U['conv_bias_relu_'+str(i)]],axis = 1)
          # State 6 : 存在slice_top,slice_conv,slice_add,slice_bottom
          else:
               SCNN_U['slice_top_'+str(i)] = tf.slice(SCNN_U['concat_'+str(i+1)],
                                                      begin = [0,0,0,0],
                                                      size = [-1,i-1,-1,-1])
               SCNN_U['slice_add_'+str(i)] = tf.slice(SCNN_U['concat_'+str(i+1)],
                                                      begin = [0,i-1,0,0],
                                                      size = [-1,1,-1,-1])
               SCNN_U['slice_conv_'+str(i)] = tf.slice(SCNN_U['concat_'+str(i+1)],
                                                      begin = [0,i,0,0],
                                                      size = [-1,1,-1,-1])
               SCNN_U['slice_bottom_'+str(i)] = tf.slice(SCNN_U['concat_'+str(i+1)],
                                                      begin = [0,i+1,0,0],
                                                      size = [-1,-1,-1,-1])
               SCNN_U['tran_bef_'+str(i)] = tf.transpose(SCNN_U['slice_conv_'+str(i)],
                                                         perm = [0,3,2,1])
               SCNN_U['pad_'+str(i)] = tf.pad(SCNN_U['tran_bef_'+str(i)],
                                              paddings = PADDING,
                                              mode='CONSTANT',
                                              constant_values=0)
               SCNN_U['conv_'+str(i)] = tf.nn.conv2d(SCNN_U['pad_'+str(i)],
                                                     filter = w,
                                                     strides = [1, 1, 1, 1],
                                                     padding = 'VALID',
                                                     use_cudnn_on_gpu=True,
                                                     data_format='NHWC',
                                                     dilations=[1, 1, 1, 1],
                                                     name=None)
               SCNN_U['conv_bias_relu_'+str(i)] = tf.nn.relu(tf.nn.bias_add(SCNN_U['conv_'+str(i)],b),
                                                             name = None)
               SCNN_U['add_'+str(i)] = tf.add(SCNN_U['conv_bias_relu_'+str(i)],
                                              SCNN_U['slice_add_'+str(i)])
               SCNN_U['concat_'+str(i)] = tf.concat(values = [SCNN_U['slice_top_'+str(i)],
                                                              SCNN_U['add_'+str(i)],
                                                              SCNN_U['conv_bias_relu_'+str(i)],
                                                              SCNN_U['slice_bottom_'+str(i)]],axis = 1)
     return SCNN_U['concat_'+str(0)]



################################################


'''
SCNN_Right 向右传递信息
'''
def SCNN_R(SCNN_R_input):
     CHANNELS = SCNN_R_input.shape[3]
     FEATURE_MAP_W = SCNN_R_input.shape[2]
     # 声明参数
     w = tf.get_variable(name = 'w_scnn_r',
                         shape = [CHANNELS,
                                  SCNN_KERNEL_LENGTH,
                                  1,
                                  CHANNELS],
                         initializer = tf.keras.initializers.glorot_normal(),
                         regularizer = tf.keras.regularizers.l2())
     b = tf.Variable(tf.zeros(CHANNELS),
                     name = "b_scnn_r")
     
     # 用于动态生成网络层
     SCNN_R = {}
   
     '''
     状态释义：
     state = 1：(when i == 0 && H > 2)
          无slice_top，存在slice_conv,slice_add,slice_bottom
     state = 2: (when i == H-2 && i > 0)
          无slice_bottom，存在slice_top,slice_conv,slice_add
     state = 3: (when i == H-1 && i > 0)
          无slice_add,slice_bottom，存在slice_top,slice_conv
     state = 4: (when H == 1)
          无slice_top,slice_add,slice_bottom，存在slice_conv
     state = 5: (when i == 0 && H == 2)
          无slice_top,slice_bottom，存在slice_conv,slice_add
     state = 6:
          存在slice_top,slice_conv,slice_add,slice_bottom
     '''
     for i in range(FEATURE_MAP_W):
          # State 1 : 无slice_top，存在slice_conv,slice_add,slice_bottom 
          if i == 0 and FEATURE_MAP_W > 2:
               SCNN_R['slice_conv_'+str(i)] = tf.slice(SCNN_R_input,
                                                      begin = [0, 0, i, 0],
                                                      size = [-1, -1, 1, -1])
               SCNN_R['slice_add_'+str(i)] = tf.slice(SCNN_R_input,
                                                      begin = [0, 0, i+1, 0],
                                                      size = [-1, -1, 1, -1])
               SCNN_R['slice_bottom_'+str(i)] = tf.slice(SCNN_R_input,
                                                      begin = [0, 0, i+2, 0],
                                                      size = [-1,-1, -1, -1])
               SCNN_R['tran_bef_'+str(i)] = tf.transpose(SCNN_R['slice_conv_'+str(i)],
                                                         perm = [0,3,1,2])
               SCNN_R['pad_'+str(i)] = tf.pad(SCNN_R['tran_bef_'+str(i)],
                                              paddings = PADDING,
                                              mode='CONSTANT',
                                              constant_values=0)
               SCNN_R['conv_'+str(i)] = tf.nn.conv2d(SCNN_R['pad_'+str(i)],
                                                     filter = w,
                                                     strides = [1, 1, 1, 1],
                                                     padding = 'VALID',
                                                     use_cudnn_on_gpu=True,
                                                     data_format='NHWC',
                                                     dilations=[1, 1, 1, 1])
               SCNN_R['conv_bias_relu_'+str(i)] = tf.nn.relu(tf.nn.bias_add(SCNN_R['conv_'+str(i)],b),
                                                             name = None)
               SCNN_R['tran_aft_'+str(i)] = tf.transpose(SCNN_R['conv_bias_relu_'+str(i)],
                                                         perm = [0,2,1,3])
               SCNN_R['add_'+str(i)] = tf.add(SCNN_R['tran_aft_'+str(i)],
                                              SCNN_R['slice_add_'+str(i)])

               SCNN_R['concat_'+str(i)] = tf.concat(values = [SCNN_R['tran_aft_'+str(i)],
                                                              SCNN_R['add_'+str(i)],
                                                              SCNN_R['slice_bottom_'+str(i)]],axis = 2)
          # State 2 : 无slice_bottom，存在slice_top,slice_conv,slice_add 
          elif i == FEATURE_MAP_W - 2 and i > 0:
               SCNN_R['slice_top_'+str(i)] = tf.slice(SCNN_R['concat_'+str(i-1)],
                                                      begin = [0, 0, 0, 0],
                                                      size = [-1, -1, i, -1])
               SCNN_R['slice_conv_'+str(i)] = tf.slice(SCNN_R['concat_'+str(i-1)],
                                                      begin = [0, 0, i, 0],
                                                      size = [-1, -1, 1, -1])
               SCNN_R['slice_add_'+str(i)] = tf.slice(SCNN_R['concat_'+str(i-1)],
                                                      begin = [0, 0, i+1, 0],
                                                      size = [-1, -1, 1, -1])
               SCNN_R['tran_bef_'+str(i)] = tf.transpose(SCNN_R['slice_conv_'+str(i)],
                                                         perm = [0,3,1,2])
               SCNN_R['pad_'+str(i)] = tf.pad(SCNN_R['tran_bef_'+str(i)],
                                              paddings = PADDING,
                                              mode='CONSTANT',
                                              constant_values=0)
               SCNN_R['conv_'+str(i)] = tf.nn.conv2d(SCNN_R['pad_'+str(i)],
                                                     filter = w,
                                                     strides = [1, 1, 1, 1],
                                                     padding = 'VALID',
                                                     use_cudnn_on_gpu=True,
                                                     data_format='NHWC',
                                                     dilations=[1, 1, 1, 1])
               SCNN_R['conv_bias_relu_'+str(i)] = tf.nn.relu(tf.nn.bias_add(SCNN_R['conv_'+str(i)],b),
                                                             name = None)
               SCNN_R['tran_aft_'+str(i)] = tf.transpose(SCNN_R['conv_bias_relu_'+str(i)],
                                                         perm = [0,2,1,3])
               SCNN_R['add_'+str(i)] = tf.add(SCNN_R['tran_aft_'+str(i)],
                                              SCNN_R['slice_add_'+str(i)])
               SCNN_R['concat_'+str(i)] = tf.concat(values = [SCNN_R['slice_top_'+str(i)],
                                                              SCNN_R['tran_aft_'+str(i)],
                                                              SCNN_R['add_'+str(i)]],axis = 2)
          # State 3 : 无slice_add,slice_bottom，存在slice_top,slice_conv 
          elif i == FEATURE_MAP_W - 1 and i > 0:
               SCNN_R['slice_top_'+str(i)] = tf.slice(SCNN_R['concat_'+str(i-1)],
                                                      begin = [0, 0, 0, 0],
                                                      size = [-1, -1, i, -1])
               SCNN_R['slice_conv_'+str(i)] = tf.slice(SCNN_R['concat_'+str(i-1)],
                                                      begin = [0, 0, i, 0],
                                                      size = [-1, -1, 1, -1])
               SCNN_R['tran_bef_'+str(i)] = tf.transpose(SCNN_R['slice_conv_'+str(i)],
                                                         perm = [0,3,1,2])
               SCNN_R['pad_'+str(i)] = tf.pad(SCNN_R['tran_bef_'+str(i)],
                                              paddings = PADDING,
                                              mode='CONSTANT',
                                              constant_values=0)
               SCNN_R['conv_'+str(i)] = tf.nn.conv2d(SCNN_R['pad_'+str(i)],
                                                     filter = w,
                                                     strides = [1, 1, 1, 1],
                                                     padding = 'VALID',
                                                     use_cudnn_on_gpu=True,
                                                     data_format='NHWC',
                                                     dilations=[1, 1, 1, 1],
                                                     name=None)
               SCNN_R['conv_bias_relu_'+str(i)] = tf.nn.relu(tf.nn.bias_add(SCNN_R['conv_'+str(i)],b),
                                                             name = None)
               SCNN_R['tran_aft_'+str(i)] = tf.transpose(SCNN_R['conv_bias_relu_'+str(i)],
                                                         perm = [0,2,1,3])
               SCNN_R['concat_'+str(i)] = tf.concat(values = [SCNN_R['slice_top_'+str(i)],
                                                              SCNN_R['tran_aft_'+str(i)]], axis = 2)
          # State 4 : 无slice_top,slice_add,slice_bottom，存在slice_conv 
          elif FEATURE_MAP_W == 1:
               SCNN_R['slice_conv_'+str(i)] = tf.slice(SCNN_R_input,
                                                      begin = [0,0,i,0],
                                                      size = [-1,-1,-1,-1])
               SCNN_R['tran_bef_'+str(i)] = tf.transpose(SCNN_R['slice_conv_'+str(i)],
                                                         perm = [0,3,1,2])
               SCNN_R['pad_'+str(i)] = tf.pad(SCNN_R['tran_bef_'+str(i)],
                                              paddings = PADDING,
                                              mode='CONSTANT',
                                              constant_values=0)
               SCNN_R['conv_'+str(i)] = tf.nn.conv2d(SCNN_R['pad_'+str(i)],
                                                     filter = w,
                                                     strides = [1, 1, 1, 1],
                                                     padding = 'VALID',
                                                     use_cudnn_on_gpu=True,
                                                     data_format='NHWC',
                                                     dilations=[1, 1, 1, 1],
                                                     name=None)
               SCNN_R['conv_bias_relu_'+str(i)] = tf.nn.relu(tf.nn.bias_add(SCNN_R['conv_'+str(i)],b),
                                                             name = None)
               SCNN_R['tran_aft_'+str(i)] = tf.transpose(SCNN_R['conv_bias_relu_'+str(i)],
                                                         perm = [0,2,1,3])
               SCNN_R['concat_'+str(i)] = SCNN_R['tran_aft_'+str(i)]
          # State 5 : 无slice_top,slice_bottom，存在slice_conv,slice_add
          elif i == 0 and FEATURE_MAP_W == 2:
               SCNN_R['slice_conv_'+str(i)] = tf.slice(SCNN_R_input,
                                                      begin = [0,0,i,0],
                                                      size = [-1,-1,1,-1])
               SCNN_R['slice_add_'+str(i)] = tf.slice(SCNN_R_input,
                                                      begin = [0,0,i+1,0],
                                                      size = [-1,-1,1,-1])
               SCNN_R['tran_bef_'+str(i)] = tf.transpose(SCNN_R['slice_conv_'+str(i)],
                                                         perm = [0,3,1,2])
               SCNN_R['pad_'+str(i)] = tf.pad(SCNN_R['tran_bef_'+str(i)],
                                              paddings = PADDING,
                                              mode='CONSTANT',
                                              constant_values=0)
               SCNN_R['conv_'+str(i)] = tf.nn.conv2d(SCNN_R['pad_'+str(i)],
                                                     filter = w,
                                                     strides = [1, 1, 1, 1],
                                                     padding = 'VALID',
                                                     use_cudnn_on_gpu=True,
                                                     data_format='NHWC',
                                                     dilations=[1, 1, 1, 1],
                                                     name=None)
               SCNN_R['conv_bias_relu_'+str(i)] = tf.nn.relu(tf.nn.bias_add(SCNN_R['conv_'+str(i)],b),
                                                             name = None)
               SCNN_R['tran_aft_'+str(i)] = tf.transpose(SCNN_R['conv_bias_relu_'+str(i)],
                                                         perm = [0,2,1,3])
               SCNN_R['add_'+str(i)] = tf.add(SCNN_R['tran_aft_'+str(i)],
                                              SCNN_R['slice_add_'+str(i)])
               SCNN_R['concat_'+str(i)] = tf.concat(values = [SCNN_R['tran_aft_'+str(i)],
                                                              SCNN_R['add_'+str(i)]],axis = 2)
          # State 6 : 存在slice_top,slice_conv,slice_add,slice_bottom
          else:
               SCNN_R['slice_top_'+str(i)] = tf.slice(SCNN_R['concat_'+str(i-1)],
                                                      begin = [0,0,0,0],
                                                      size = [-1,-1,i,-1])
               SCNN_R['slice_conv_'+str(i)] = tf.slice(SCNN_R['concat_'+str(i-1)],
                                                      begin = [0,0,i,0],
                                                      size = [-1,-1,1,-1])
               SCNN_R['slice_add_'+str(i)] = tf.slice(SCNN_R['concat_'+str(i-1)],
                                                      begin = [0,0,i+1,0],
                                                      size = [-1,-1,1,-1])
               SCNN_R['slice_bottom_'+str(i)] = tf.slice(SCNN_R['concat_'+str(i-1)],
                                                      begin = [0,0,i+2,0],
                                                      size = [-1,-1,-1,-1])
               SCNN_R['tran_bef_'+str(i)] = tf.transpose(SCNN_R['slice_conv_'+str(i)],
                                                         perm = [0,3,1,2])
               SCNN_R['pad_'+str(i)] = tf.pad(SCNN_R['tran_bef_'+str(i)],
                                              paddings = PADDING,
                                              mode='CONSTANT',
                                              constant_values=0)
               SCNN_R['conv_'+str(i)] = tf.nn.conv2d(SCNN_R['pad_'+str(i)],
                                                     filter = w,
                                                     strides = [1, 1, 1, 1],
                                                     padding = 'VALID',
                                                     use_cudnn_on_gpu=True,
                                                     data_format='NHWC',
                                                     dilations=[1, 1, 1, 1],
                                                     name=None)
               SCNN_R['conv_bias_relu_'+str(i)] = tf.nn.relu(tf.nn.bias_add(SCNN_R['conv_'+str(i)],b),
                                                             name = None)
               SCNN_R['tran_aft_'+str(i)] = tf.transpose(SCNN_R['conv_bias_relu_'+str(i)],
                                                         perm = [0,2,1,3])
               SCNN_R['add_'+str(i)] = tf.add(SCNN_R['tran_aft_'+str(i)],
                                              SCNN_R['slice_add_'+str(i)])
               SCNN_R['concat_'+str(i)] = tf.concat(values = [SCNN_R['slice_top_'+str(i)],
                                                              SCNN_R['tran_aft_'+str(i)],
                                                              SCNN_R['add_'+str(i)],
                                                              SCNN_R['slice_bottom_'+str(i)]],axis = 2)

     return SCNN_R['concat_'+str(FEATURE_MAP_W-1)]



'''
SCNN_Left 向左传递信息
'''
def SCNN_L(SCNN_L_input):
     CHANNELS = SCNN_L_input.shape[3]
     FEATURE_MAP_W = SCNN_L_input.shape[2]
     # 声明参数
     w = tf.get_variable(name = 'w_scnn_l',
                         shape = [CHANNELS,
                                  SCNN_KERNEL_LENGTH,
                                  1,
                                  CHANNELS],
                         initializer = tf.keras.initializers.glorot_normal(),
                         regularizer = tf.keras.regularizers.l2())
     b = tf.Variable(tf.zeros(CHANNELS),
                     name = "b_scnn_l")
     
     # 用于动态生成网络层
     SCNN_L = {}

     '''
     状态释义：
     state = 1：(when i == H-1 && H > 2)
          无slice_bottom,存在slice_conv,slice_add,slice_top
     state = 2: (when i == 1 && H > 2)
          无slice_top，存在slice_bottom,slice_conv,slice_add
     state = 3: (when i == 0 && H > 1)
          无slice_add,slice_top，存在slice_bottom,slice_conv
     state = 4: (when H == 1)
          无slice_top,slice_add,slice_bottom，存在slice_conv
     state = 5: (when i == H-1 && H == 2)
          无slice_top,slice_bottom，存在slice_conv,slice_add
     state = 6:
          存在slice_top,slice_conv,slice_add,slice_bottom
     '''
     for i in range(FEATURE_MAP_W-1,-1,-1):
          # State 1 : 无slice_bottom,存在slice_conv,slice_add,slice_top 
          if i == FEATURE_MAP_W-1 and FEATURE_MAP_W > 2:
               SCNN_L['slice_conv_'+str(i)] = tf.slice(SCNN_L_input,
                                                      begin = [0, 0, i, 0],
                                                      size = [-1, -1, 1, -1])
               SCNN_L['slice_add_'+str(i)] = tf.slice(SCNN_L_input,
                                                      begin = [0, 0, i-1, 0],
                                                      size = [-1, -1, 1, -1])
               SCNN_L['slice_top_'+str(i)] = tf.slice(SCNN_L_input,
                                                      begin = [0, 0, 0, 0],
                                                      size = [-1, -1, i-1, -1])
               SCNN_L['tran_bef_'+str(i)] = tf.transpose(SCNN_L['slice_conv_'+str(i)],
                                                         perm = [0,3,1,2])
               SCNN_L['pad_'+str(i)] = tf.pad(SCNN_L['tran_bef_'+str(i)],
                                              paddings = PADDING,
                                              mode='CONSTANT',
                                              constant_values=0)
               SCNN_L['conv_'+str(i)] = tf.nn.conv2d(SCNN_L['pad_'+str(i)],
                                                     filter = w,
                                                     strides = [1, 1, 1, 1],
                                                     padding = 'VALID',
                                                     use_cudnn_on_gpu=True,
                                                     data_format='NHWC',
                                                     dilations=[1, 1, 1, 1])
               SCNN_L['conv_bias_relu_'+str(i)] = tf.nn.relu(tf.nn.bias_add(SCNN_L['conv_'+str(i)],b),
                                                             name = None)
               SCNN_L['conv_bias_relu_'+str(i)] = tf.transpose(SCNN_L['conv_bias_relu_'+str(i)],
                                                         perm = [0,2,1,3])
               SCNN_L['add_'+str(i)] = tf.add(SCNN_L['conv_bias_relu_'+str(i)],
                                              SCNN_L['slice_add_'+str(i)])
               SCNN_L['concat_'+str(i)] = tf.concat(values = [SCNN_L['slice_top_'+str(i)],
                                                              SCNN_L['add_'+str(i)],
                                                              SCNN_L['conv_bias_relu_'+str(i)]],axis = 2)
          # State 2 : 无slice_top，存在slice_bottom,slice_conv,slice_add 
          elif i == 1 and FEATURE_MAP_W > 2:
               SCNN_L['slice_add_'+str(i)] = tf.slice(SCNN_L['concat_'+str(i+1)],
                                                      begin = [0, 0, 0, 0],
                                                      size = [-1, -1, i, -1])
               SCNN_L['slice_conv_'+str(i)] = tf.slice(SCNN_L['concat_'+str(i+1)],
                                                      begin = [0, 0, i, 0],
                                                      size = [-1, -1, 1, -1])
               SCNN_L['slice_bottom_'+str(i)] = tf.slice(SCNN_L['concat_'+str(i+1)],
                                                      begin = [0, 0, i+1, 0],
                                                      size = [-1, -1, -1, -1])
               SCNN_L['tran_bef_'+str(i)] = tf.transpose(SCNN_L['slice_conv_'+str(i)],
                                                         perm = [0,3,1,2])
               SCNN_L['pad_'+str(i)] = tf.pad(SCNN_L['tran_bef_'+str(i)],
                                              paddings = PADDING,
                                              mode='CONSTANT',
                                              constant_values=0)
               SCNN_L['conv_'+str(i)] = tf.nn.conv2d(SCNN_L['pad_'+str(i)],
                                                     filter = w,
                                                     strides = [1, 1, 1, 1],
                                                     padding = 'VALID',
                                                     use_cudnn_on_gpu=True,
                                                     data_format='NHWC',
                                                     dilations=[1, 1, 1, 1])
               SCNN_L['conv_bias_relu_'+str(i)] = tf.nn.relu(tf.nn.bias_add(SCNN_L['conv_'+str(i)],b),
                                                             name = None)
               SCNN_L['conv_bias_relu_'+str(i)] = tf.transpose(SCNN_L['conv_bias_relu_'+str(i)],
                                                         perm = [0,2,1,3])
               SCNN_L['add_'+str(i)] = tf.add(SCNN_L['conv_bias_relu_'+str(i)],
                                              SCNN_L['slice_add_'+str(i)])
               SCNN_L['concat_'+str(i)] = tf.concat(values = [SCNN_L['add_'+str(i)],
                                                              SCNN_L['conv_bias_relu_'+str(i)],
                                                              SCNN_L['slice_bottom_'+str(i)]],axis = 2)
          # State 3 : 无slice_add,slice_top，存在slice_bottom,slice_conv
          elif i == 0 and FEATURE_MAP_W > 1:
               SCNN_L['slice_conv_'+str(i)] = tf.slice(SCNN_L['concat_'+str(i+1)],
                                                      begin = [0, 0, i, 0],
                                                      size = [-1, -1, 1, -1])
               SCNN_L['slice_bottom_'+str(i)] = tf.slice(SCNN_L['concat_'+str(i+1)],
                                                      begin = [0, 0, i+1, 0],
                                                      size = [-1, -1, -1, -1])
               SCNN_L['tran_bef_'+str(i)] = tf.transpose(SCNN_L['slice_conv_'+str(i)],
                                                         perm = [0,3,1,2])
               SCNN_L['pad_'+str(i)] = tf.pad(SCNN_L['tran_bef_'+str(i)],
                                              paddings = PADDING,
                                              mode='CONSTANT',
                                              constant_values=0)
               SCNN_L['conv_'+str(i)] = tf.nn.conv2d(SCNN_L['pad_'+str(i)],
                                                     filter = w,
                                                     strides = [1, 1, 1, 1],
                                                     padding = 'VALID',
                                                     use_cudnn_on_gpu=True,
                                                     data_format='NHWC',
                                                     dilations=[1, 1, 1, 1],
                                                     name=None)
               SCNN_L['conv_bias_relu_'+str(i)] = tf.nn.relu(tf.nn.bias_add(SCNN_L['conv_'+str(i)],b),
                                                             name = None)
               SCNN_L['conv_bias_relu_'+str(i)] = tf.transpose(SCNN_L['conv_bias_relu_'+str(i)],
                                                         perm = [0,2,1,3])
               SCNN_L['concat_'+str(i)] = tf.concat(values = [SCNN_L['conv_bias_relu_'+str(i)],
                                                              SCNN_L['slice_bottom_'+str(i)]], axis = 2)
          # State 4 : 无slice_top,slice_add,slice_bottom，存在slice_conv 
          elif FEATURE_MAP_W == 1:
               SCNN_L['slice_conv_'+str(i)] = tf.slice(SCNN_L_input,
                                                      begin = [0,0,i,0],
                                                      size = [-1,-1,-1,-1])
               SCNN_L['tran_bef_'+str(i)] = tf.transpose(SCNN_L['slice_conv_'+str(i)],
                                                         perm = [0,3,1,2])
               SCNN_L['pad_'+str(i)] = tf.pad(SCNN_L['tran_bef_'+str(i)],
                                              paddings = PADDING,
                                              mode='CONSTANT',
                                              constant_values=0)
               SCNN_L['conv_'+str(i)] = tf.nn.conv2d(SCNN_L['pad_'+str(i)],
                                                     filter = w,
                                                     strides = [1, 1, 1, 1],
                                                     padding = 'VALID',
                                                     use_cudnn_on_gpu=True,
                                                     data_format='NHWC',
                                                     dilations=[1, 1, 1, 1],
                                                     name=None)
               SCNN_L['conv_bias_relu_'+str(i)] = tf.nn.relu(tf.nn.bias_add(SCNN_L['conv_'+str(i)],b),
                                                             name = None)
               SCNN_L['conv_bias_relu_'+str(i)] = tf.transpose(SCNN_L['conv_bias_relu_'+str(i)],
                                                         perm = [0,2,1,3])
               SCNN_L['concat_'+str(i)] = SCNN_L['conv_bias_relu_'+str(i)]
          # State 5 : 无slice_top,slice_bottom，存在slice_conv,slice_add
          elif i == FEATURE_MAP_W-1 and FEATURE_MAP_W == 2:
               SCNN_L['slice_add_'+str(i)] = tf.slice(SCNN_L_input,
                                                      begin = [0,0,i,0],
                                                      size = [-1,-1,1,-1])
               SCNN_L['slice_conv_'+str(i)] = tf.slice(SCNN_L_input,
                                                      begin = [0,0,i+1,0],
                                                      size = [-1,-1,1,-1])
               SCNN_L['tran_bef_'+str(i)] = tf.transpose(SCNN_L['slice_conv_'+str(i)],
                                                         perm = [0,3,1,2])
               SCNN_L['pad_'+str(i)] = tf.pad(SCNN_L['tran_bef_'+str(i)],
                                              paddings = PADDING,
                                              mode='CONSTANT',
                                              constant_values=0)
               SCNN_L['conv_'+str(i)] = tf.nn.conv2d(SCNN_L['pad_'+str(i)],
                                                     filter = w,
                                                     strides = [1, 1, 1, 1],
                                                     padding = 'VALID',
                                                     use_cudnn_on_gpu=True,
                                                     data_format='NHWC',
                                                     dilations=[1, 1, 1, 1],
                                                     name=None)
               SCNN_L['conv_bias_relu_'+str(i)] = tf.nn.relu(tf.nn.bias_add(SCNN_L['conv_'+str(i)],b),
                                                             name = None)
               SCNN_L['conv_bias_relu_'+str(i)] = tf.transpose(SCNN_L['conv_bias_relu_'+str(i)],
                                                         perm = [0,2,1,3])
               SCNN_L['add_'+str(i)] = tf.add(SCNN_L['conv_bias_relu_'+str(i)],
                                              SCNN_L['slice_add_'+str(i)])
               SCNN_L['concat_'+str(i)] = tf.concat(values = [SCNN_L['add_'+str(i)],
                                                              SCNN_L['conv_bias_relu_'+str(i)]],axis = 2)
          # State 6 : 存在slice_top,slice_conv,slice_add,slice_bottom
          else:
               SCNN_L['slice_top_'+str(i)] = tf.slice(SCNN_L['concat_'+str(i+1)],
                                                      begin = [0,0,0,0],
                                                      size = [-1,-1,i-1,-1])
               SCNN_L['slice_add_'+str(i)] = tf.slice(SCNN_L['concat_'+str(i+1)],
                                                      begin = [0,0,i-1,0],
                                                      size = [-1,-1,1,-1])
               SCNN_L['slice_conv_'+str(i)] = tf.slice(SCNN_L['concat_'+str(i+1)],
                                                      begin = [0,0,i,0],
                                                      size = [-1,-1,1,-1])
               SCNN_L['slice_bottom_'+str(i)] = tf.slice(SCNN_L['concat_'+str(i+1)],
                                                      begin = [0,0,i+1,0],
                                                      size = [-1,-1,-1,-1])
               SCNN_L['tran_bef_'+str(i)] = tf.transpose(SCNN_L['slice_conv_'+str(i)],
                                                         perm = [0,3,1,2])
               SCNN_L['pad_'+str(i)] = tf.pad(SCNN_L['tran_bef_'+str(i)],
                                              paddings = PADDING,
                                              mode='CONSTANT',
                                              constant_values=0)
               SCNN_L['conv_'+str(i)] = tf.nn.conv2d(SCNN_L['pad_'+str(i)],
                                                     filter = w,
                                                     strides = [1, 1, 1, 1],
                                                     padding = 'VALID',
                                                     use_cudnn_on_gpu=True,
                                                     data_format='NHWC',
                                                     dilations=[1, 1, 1, 1],
                                                     name=None)
               SCNN_L['conv_bias_relu_'+str(i)] = tf.nn.relu(tf.nn.bias_add(SCNN_L['conv_'+str(i)],b),
                                                             name = None)
               SCNN_L['conv_bias_relu_'+str(i)] = tf.transpose(SCNN_L['conv_bias_relu_'+str(i)],
                                                         perm = [0,2,1,3])
               SCNN_L['add_'+str(i)] = tf.add(SCNN_L['conv_bias_relu_'+str(i)],
                                              SCNN_L['slice_add_'+str(i)])
               SCNN_L['concat_'+str(i)] = tf.concat(values = [SCNN_L['slice_top_'+str(i)],
                                                              SCNN_L['add_'+str(i)],
                                                              SCNN_L['conv_bias_relu_'+str(i)],
                                                              SCNN_L['slice_bottom_'+str(i)]],axis = 2)
     return SCNN_L['concat_'+str(0)]




#%% 定义网络
inputs = tf.keras.Input(shape = (6,300,1),
                        #batch_size = BATCH_SIZE,
                        name = 'Input_Layer',
                        dtype = tf.float32)

with tf.name_scope('SCNN_D'):
    SCNN_D_out = tf.keras.layers.Lambda(SCNN_D)(inputs)
with tf.name_scope('SCNN_U'):
    SCNN_U_out = tf.keras.layers.Lambda(SCNN_U)(inputs)
with tf.name_scope('SCNN_R'):
    SCNN_R_out = tf.keras.layers.Lambda(SCNN_R)(inputs)
with tf.name_scope('SCNN_L'):
    SCNN_L_out = tf.keras.layers.Lambda(SCNN_L)(inputs)


concat = tf.keras.layers.concatenate([inputs,SCNN_D_out,SCNN_U_out,SCNN_R_out,SCNN_L_out],axis = -1)

print(concat.shape)
    
conv_1 = tf.keras.layers.Conv2D(filters = CONV1_FILTERS,
                                kernel_size = (3,3),
                                strides = (1,1),
                                padding = 'same',
                                data_format = 'channels_last',
                                activation = 'relu',
                                use_bias = True,
                                kernel_initializer = tf.keras.initializers.glorot_normal(),
                                bias_initializer = tf.zeros_initializer(),
                                kernel_regularizer = tf.keras.regularizers.l2(),
                                bias_regularizer = tf.keras.regularizers.l2())(concat)

maxpool_1 = tf.keras.layers.MaxPool2D(pool_size = (2,2),
                                      strides = (2,2),
                                      padding = 'valid',
                                      data_format = 'channels_last')(conv_1)

conv_2 = tf.keras.layers.Conv2D(filters = CONV2_FILTERS,
                                kernel_size = (3,3),
                                strides = (1,1),
                                padding = 'same',
                                data_format = 'channels_last',
                                activation = 'relu',
                                use_bias = True,
                                kernel_initializer = tf.keras.initializers.glorot_normal(),
                                bias_initializer = tf.zeros_initializer(),
                                kernel_regularizer = tf.keras.regularizers.l2(),
                                bias_regularizer = tf.keras.regularizers.l2())(maxpool_1)
maxpool_2 = tf.keras.layers.MaxPool2D(pool_size = (2,2),
                                      strides = (2,2),
                                      padding = 'valid',
                                      data_format = 'channels_last')(conv_2)


print("888888888888888888888")


fc_input = tf.keras.layers.Flatten()(maxpool_2)

fc_1 = tf.keras.layers.Dense(units = 1024,
                             activation = 'tanh',
                             use_bias = True,
                             kernel_initializer = tf.keras.initializers.glorot_normal(),
                             bias_initializer = tf.zeros_initializer(),
                             kernel_regularizer = tf.keras.regularizers.l2(),
                             bias_regularizer = tf.keras.regularizers.l2())(fc_input)

dropout_1 = tf.keras.layers.Dropout(0.1)(fc_1)

fc_2 = tf.keras.layers.Dense(units = 512,
                             activation = 'tanh',
                             use_bias = True,
                             kernel_initializer = tf.keras.initializers.glorot_normal(),
                             bias_initializer = tf.zeros_initializer(),
                             kernel_regularizer = tf.keras.regularizers.l2(),
                             bias_regularizer = tf.keras.regularizers.l2())(dropout_1)

dropout_2 = tf.keras.layers.Dropout(0.1)(fc_2)

fc_3 = tf.keras.layers.Dense(units = 256,
                             activation = 'tanh',
                             use_bias = True,
                             kernel_initializer = tf.keras.initializers.glorot_normal(),
                             bias_initializer = tf.zeros_initializer(),
                             kernel_regularizer = tf.keras.regularizers.l2(),
                             bias_regularizer = tf.keras.regularizers.l2())(dropout_2)

dropout_3 = tf.keras.layers.Dropout(0.1)(fc_3)


logits = tf.keras.layers.Dense(units = 5,
                             activation = 'softmax',
                             use_bias = True,
                             kernel_initializer = tf.keras.initializers.glorot_normal(),
                             bias_initializer = tf.zeros_initializer(),
                             kernel_regularizer = tf.keras.regularizers.l2(),
                             bias_regularizer = tf.keras.regularizers.l2())(dropout_3)

model = tf.keras.Model(inputs = inputs,outputs = logits)

model.compile(optimizer = 'Adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])
tensorboard = tf.keras.callbacks.TensorBoard(log_dir='./logs', 
                                             histogram_freq=0, 
                                             write_graph=True, 
                                             write_images=True,
                                             write_grads = True)

#%%
print("正在训练网络，请耐心等候......")
startTime = time.clock()
history = model.fit(x = x,
          y = y_onehot,
          batch_size = BATCH_SIZE,
          epochs=20,
          verbose=1,
          callbacks=[tensorboard],
          validation_split=0.03,
          validation_data=None,
          shuffle=True,
          class_weight=None,
          sample_weight=None,
          initial_epoch=0,
          steps_per_epoch= None,
          validation_steps= None)

endTime = time.clock()
# 注意，这里的时间window和linux不同
print("网络训练已完成 耗时%f 秒"%((float)(endTime - startTime)/10))

# ## 绘制模型的结构图
# tf.keras.utils.plot_model(model,
#                           to_file='model.png',
#                           show_shapes=True,
#                           show_layer_names=True)

'''注意：sklearn中的标签都不是稀疏的'''
import sklearn
from sklearn.metrics import classification_report
predict = model.predict(x_test)
print("测试集预测结果：")
print(classification_report(np.argmax(y_test,axis = 1),np.argmax(predict,axis = 1)))
# 打印混淆矩阵
print("测试集混淆矩阵：")
print(sklearn.metrics.confusion_matrix(np.argmax(y_test,axis = 1),np.argmax(predict,axis = 1)))

##%%
## from keras.models import load_model
# 保存模型
model.save('SCNN_Model.h5')



(?, 6, 300, 5)
888888888888888888888
正在训练网络，请耐心等候......
Train on 900 samples, validate on 28 samples
Epoch 1/20
900/900 [==============================] - 26s 29ms/step - loss: 19.9503 - acc: 0.5411 - val_loss: 17.2016 - val_acc: 0.6786
Epoch 2/20
900/900 [==============================] - 2s 2ms/step - loss: 16.1712 - acc: 0.8156 - val_loss: 14.4797 - val_acc: 0.8929
Epoch 3/20
900/900 [==============================] - 2s 2ms/step - loss: 13.6470 - acc: 0.9033 - val_loss: 12.3163 - val_acc: 0.8571
Epoch 4/20
900/900 [==============================] - 2s 2ms/step - loss: 11.6345 - acc: 0.8944 - val_loss: 10.4280 - val_acc: 0.9643
Epoch 5/20
900/900 [==============================] - 2s 2ms/step - loss: 9.9376 - acc: 0.9467 - val_loss: 8.9365 - val_acc: 1.0000
Epoch 6/20
900/900 [==============================] - 2s 2ms/step - loss: 8.5350 - acc: 0.9678 - val_loss: 8.0440 - val_acc: 0.8214
Epoch 7/20
900/900 [==============================] - 2s 2ms/step - loss: 7.5059 - acc: 0.9267 - 

Epoch 13/20
900/900 [==============================] - 2s 2ms/step - loss: 3.6630 - acc: 0.9889 - val_loss: 3.3885 - val_acc: 1.0000
Epoch 14/20
900/900 [==============================] - 2s 2ms/step - loss: 3.2719 - acc: 0.9911 - val_loss: 3.0601 - val_acc: 0.9643
Epoch 15/20
900/900 [==============================] - 2s 2ms/step - loss: 2.9810 - acc: 0.9711 - val_loss: 2.7303 - val_acc: 1.0000
Epoch 16/20
900/900 [==============================] - 2s 2ms/step - loss: 2.6356 - acc: 0.9933 - val_loss: 2.4740 - val_acc: 1.0000
Epoch 17/20
900/900 [==============================] - 2s 2ms/step - loss: 2.4756 - acc: 0.9556 - val_loss: 2.2492 - val_acc: 1.0000
Epoch 18/20
900/900 [==============================] - 2s 2ms/step - loss: 2.1987 - acc: 0.9833 - val_loss: 2.0719 - val_acc: 1.0000
Epoch 19/20
900/900 [==============================] - 2s 2ms/step - loss: 2.0109 - acc: 0.9856 - val_loss: 1.8645 - val_acc: 1.0000
Epoch 20/20
900/900 [==============================] - 2s 2ms/step - 